# Ozone vertical profile

## Import libraries

In [ ]:
import collections

import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import download

## Define request

In [ ]:
collection_id = "satellite-ozone-v1"
common_request = {
    "processing_level": "level_3",
    "variable": "mole_concentration_of_ozone_in_air",
    "vertical_aggregation": "vertical_profiles_from_limb_sensors",
}
sensor_keys = ["sensor", "version"]
limb_requests = [
    {
        "sensor": "gomos",
        "version": "v0001",
        "start": "2002-08",
        "stop": "2011-12",
        "skip": [
            "2005-02",
            "2005-03",
            "2005-04",
            "2005-05",
            "2005-06",
            "2011-11",
        ],
    },
    {
        "sensor": "mipas",
        "version": "v0001",
        "start": "2002-07",
        "stop": "2012-04",
        "skip": [
            "2004-04",
            "2004-05",
            "2004-06",
            "2004-07",
            "2004-08",
            "2004-09",
            "2004-10",
            "2004-11",
            "2004-12",
            "2005-09",
            "2005-10",
            "2005-11",
            "2006-02",
            "2006-04",
        ],
    },
    {
        "sensor": "sage_2",
        "version": "v0001",
        "start": "1984-10",
        "stop": "2005-08",
        "skip": [
            "2000-08",
            "2000-09",
            "2000-10",
        ],
    },
    {
        "sensor": "sciamachy",
        "version": "v0001",
        "start": "2002-08",
        "stop": "2012-03",
    },
    {
        "sensor": "smr",
        "version": "v0001",
        "start": "2001-07",
        "stop": "2014-08",
        "skip": [
            "2001-09",
        ],
    },
    {
        "sensor": "cmzm",
        "version": "v0008",
        "start": "1984-10",
        "stop": "2022-12",
        "rename": {
            "merged_ozone_concentration": "ozone_concentration",
            "uncertainty_of_merged_ozone_concentration": "mean_uncertainty_estimate",
        },
    },
]
assert all(
    set(sensor_keys) | {"start", "stop"} <= set(request) for request in limb_requests
)

## Download

In [ ]:
datasets = collections.defaultdict(list)
for limb_request in limb_requests:
    request = common_request | {key: limb_request[key] for key in sensor_keys}
    sensor = "-".join([request[sensor_key] for sensor_key in sensor_keys])
    print(f"{sensor = }")
    for time in pd.date_range(limb_request["start"], limb_request["stop"], freq="1MS"):
        if time.strftime("%Y-%m") in limb_request.get("skip", []):
            continue
        ds = download.download_and_transform(
            collection_id,
            request | {"year": time.strftime("%Y"), "month": time.strftime("%m")},
            quiet=True,
        )
        ds = ds.rename(limb_request.get("rename", {}))
        ds = ds.squeeze("source", drop=True).expand_dims(sensor=[sensor], time=[time])
        datasets[sensor].append(ds)
ds = xr.concat(
    [xr.concat(v, "time") for k, v in datasets.items()],
    "sensor",
)

## Quick and dirty plot

In [ ]:
time = time = "2005-08"
ds["ozone_concentration"].sel(time=time).plot(col="sensor", col_wrap=3)